## Scale up model size
---
In previous notebooks, we downloaded and extracted our own Swedish raw text with `Lab2-1_acquiring_data.ipynb`; practiced filtering, cleaning and deduplicating the raw text data with `Lab2-2_SentenceBoundary_and_Deduplicate.ipynb` ; trained our own GPTBPETokenizer using the raw Swedish text with `Lab2-3_train_own_GPT2BPETokenizer.ipynb`; and converted the raw text to mmap format integrating a custom sentence-splitter in `Lab2-4_customize_process2mmap.ipynb`.

We have learned all the essential components in order to customize Megatron-LM's default workflow  accommodating to specific language needs (in this case, Swedish). The obvious next step is to train the Megatron-LM GPT model with the processed Swedish data.
However, constrainedt by how much compute resources one could get, i.e. the number of GPUs available for the training job, there is an upper limit of how big a model you can train.

We will test ou t what size model we could train with 2 X A100 GPUs 40GB, by presenting a challenge!

## **Challenge ** - Go big or go home !

- Constraints : 
    - 2 x A100 GPUs 40G is allocated for this challenge.
    - Only the parameters in the **##### Begin/End of modifiable blocks #####** are allowed to be changed.
    - Avoid OOM !
    - Training run must be finished and the checkpoint must be saved successfully.

- Task : 
        Given the above constraints, train the biggest GPT model as possible.

- Winning criteria : The biggest model wins!

Note 1: Post the parameters you changed into the **##### Begin/End of modifiable blocks #####**  on bootcamp's slack channels for verification.

Note 2: We purposefully turned-off nsys profiling in this challenge, because calling nsys profiling will introduce a small overhead, which will impact the maximum achievable model size.

Go directly to the code block and modify the training configuration, click here to <a href="./Lab2-5_run_Megatron_with_varying_config.ipynb#MODIFY_CELL">Jump to Code Cell and Modify Training Config</a> 


**Hint** :
Use the knowledge gained from `Lab1-6_Observe_GPT_runs_vs_performance.ipynb`, especially the section with video demonstrating how to do live profiling during a live training run.

Modify and rerun the code blocks below to obtain an even bigger GPT model. 


<a id="MODIFY_CELL"></a>
<a href="./Lab2-5_run_Megatron_with_varying_config.ipynb#Rerun_Cell">Jump to ReRun Cell</a> 

<a id="MODIFY_CELL"></a>

Always clean the checkpoint folder to ensure training starts from scratch.

In [ ]:
!rm -fr ../sv_ckpt/* 
!rm -fr ../dataset/SV/*.npy

In [ ]:
%%writefile ./Megatron-LM/SV_GPT_goingBIG.sh
# Copyright (c) 2020 NVIDIA Corporation.  All rights reserved.
# Copyright (c) 2020 NVIDIA Corporation.  All rights reserved.
GPUS_PER_NODE=2 # <--- remember to change the number of GPUs you actually have in your system
# Change for multinode config
MASTER_ADDR=localhost
MASTER_PORT=6000
NNODES=1 #<-- currently we are using 1 node multigpus
NODE_RANK=0
WORLD_SIZE=2 # <--- remember to change the number of GPUs you actually have in your system

### modify this section to point the file to its own path 
CHECKPOINT_PATH='../sv_ckpt/' ## modify this path if you customize it 
DATA_PATH='../dataset/SV/webnyheter2013_32kvocab_text_document' ## modify this path if you customize it 
VOCAB_FILE='../dataset/SV/32k/vocab.json' ## modify this path if you customize it 
MERGE_FILE='../dataset/SV/32k/merges.txt' ## modify this path if you customize it 
PROFILE_OUTPUT_PATH='../profiles/SV/nsys_improved2' # modify this to your own profile path

################   Beginning of modifiable section    ####################
TENSOR_MP_SIZE=<FILL_IN>
PIPELINE_MP_SIZE=<FILL_IN>
NUM_LYS=<FILL_IN>
HIDDEN_SIZE=<FILL_IN>
NUM_ATTN_HEADS=<FILL_IN>
SEQ_LEN=<FILL_IN>
MAX_POS_EM=<FILL_IN>
MICRO_BZ=<FILL_IN>
GLOBAL_BZ=<FILL_IN>

##############   end of modifiable sectio, do NOT modify anything below this line    ####################

export OMP_NUM_THREADS=1
DISTRIBUTED_ARGS="--nproc_per_node $GPUS_PER_NODE --nnodes $NNODES --node_rank $NODE_RANK --master_addr $MASTER_ADDR --master_port $MASTER_PORT"

## for nsys run
#nsys profile --stats=false --force-overwrite=true --duration=300 --trace=cudnn,cuda,osrt,nvtx -o $PROFILE_OUTPUT_PATH \
python -m torch.distributed.launch $DISTRIBUTED_ARGS \
    ./Megatron-LM/pretrain_gpt.py \
       --tensor-model-parallel-size ${TENSOR_MP_SIZE} \
       --pipeline-model-parallel-size ${PIPELINE_MP_SIZE} \
       --num-layers ${NUM_LYS} \
       --hidden-size ${HIDDEN_SIZE} \
       --num-attention-heads ${NUM_ATTN_HEADS} \
       --micro-batch-size ${MICRO_BZ} \
       --global-batch-size ${GLOBAL_BZ} \
       --seq-length ${SEQ_LEN} \
       --max-position-embeddings ${MAX_POS_EM} \
       --train-samples 100 \
       --save ${CHECKPOINT_PATH} \
       --load ${CHECKPOINT_PATH} \
       --data-path ${DATA_PATH} \
       --vocab-file ${VOCAB_FILE} \
       --merge-file ${MERGE_FILE} \
       --data-impl mmap \
       --split 949,50,1 \
       --distributed-backend nccl \
       --lr 0.00015 \
       --lr-decay-style cosine \
       --min-lr 1.0e-5 \
       --weight-decay 1e-2 \
       --clip-grad 1.0 \
       --lr-warmup-fraction .01 \
       --checkpoint-activations \
       --log-interval 10 \
       --save-interval 100 \
       --eval-interval 200 \
       --eval-iters 10 \
       --fp16

Check how big your model is by modify the parameters in the [params_cnt.sh](./params_cnt.sh) to match the training parames above.

I got 1.6 illion ! :)  what about you ?

Modify the [params count](./params_cnt.sh) according to your training configuration.

After modification, run the below bash script to obtain the model size.

In [ ]:
!bash params_cnt.sh 

Below is an example of expected outputs:
    
        1 <-- One could get different number depend on your training config
        1678049280 <-- One could get different number depend on your training config


Re-run this cell below to get an even bigger GPT model

Remember to modify the [params count](./params_cnt.sh) to check how big your model is.

Jump back and edit the SV_GPT_goingBIG.sh, click here to 
<a href="./Lab2-5_run_Megatron_with_varying_config.ipynb#MODIFY_CELL">Jump back to modify and overwrite SV_GPT_goingBIG.sh </a> 
<a id="Rerun_Cell"></a>

In [ ]:
!bash ./Megatron-LM/SV_GPT_goingBIG.sh

Below is an example of expected outputs:

        > elapsed time for building blendable dataset indices: 0.00 (sec)
        > finished creating GPT datasets ...
        [after dataloaders are built] datetime: 2021-09-15 11:55:58 
        done with setup ...
        training ...
        time (ms) | model-and-optimizer-setup: 929.42 | train/valid/test-data-iterators-setup: 1004.53
        [after training is done] datetime: 2021-09-15 11:55:58 
        ------------------------------------------------------------------------------------------------------------------
         validation loss at the end of training for val data | lm loss value: 1.171452E+01 | lm loss PPL: 1.223352E+05 | 
        ------------------------------------------------------------------------------------------------------------------
        Evaluating iter 10/10
        -------------------------------------------------------------------------------------------------------------------
         validation loss at the end of training for test data | lm loss value: 1.171400E+01 | lm loss PPL: 1.222719E+05 | 
        -------------------------------------------------------------------------------------------------------------------

---

## Links and Resources
Don't forget to read more on [Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165.pdf) and [Efficient Large-Scale Language Model Training on GPU Clusters](https://arxiv.org/pdf/2104.04473.pdf).

-----
## <p style="text-align:center;border:3px; padding: 1em"> <a href=../Start_Here.ipynb>HOME</a></p>

-----


## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). 